# MongoDB Atlas Search: More like this

## Setup

In [ ]:
# import our libraries, instantiate our classes
import sys
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pprint

# Load environment variables from a .env file, overriding any existing variables in the environment.
# This is useful for not cluttering the code with sensitive information like API keys and database URIs.
load_dotenv(override=True)

MONGO_URI = os.environ["MONGO_URI"]

client = MongoClient(MONGO_URI)['sample_mflix']

pp = pprint.PrettyPrinter(indent=1)

#### Atlas Search Index

```json

{
  "mappings": {
    "dynamic": true
  }
}

```

### Let's fetch an example document

In [ ]:
# fetch a sample document, in this case, the movie "The Godfather"
one_document_query = [
    {'$match':{"title":"The Godfather"}},
    {'$project':{
        "genres":1,
        "plot":1,
        "_id":0
    }},
    {"$limit":1}
]

result = list(client['movies'].aggregate(one_document_query))
pp.pprint(result)

#### Now, we provide the result above as a reference document to the MoreLikeThis query:

In [ ]:
# Construct 
more_like_this_query = [
    {
        '$search': {
            'index': 'default', 
            'moreLikeThis': {
                'like': result
            }
        }
    },
    {
        '$project':{
            "genres":1,
            "plot":1,
            "title":1,
            "_id":0,
            'score': {
                '$meta': 'searchScore'
            }
        }
    },
    {'$limit':10}
]

pp.pprint(list(client['movies'].aggregate(more_like_this_query)))